<div style="background:#F5F7FA; height:100px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Want to do more?</span><span style="border: 1px solid #3d70b2;padding: 15px;float:right;margin-right:40px; color:#3d70b2; "><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
<span style="color:#5A6872;"> Try out this notebook with your free trial of IBM Watson Studio.</span>
</div>

<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predict Iris Species with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps which describe from storing sample Predictive Model Markup Language (PMML) model to starting scoring new data. 

Some familiarity with python is helpful. This notebook uses Python 3.5 with Spark 2.1.

You will use an **Iris** data set, which details measurements of iris perianth. Use the details of this data set to predict the iris species.

## Learning goals

The learning goals of this notebook are:

-  Work with the Watson Machine Learning (WML) instance
-  Deploy the PMML model online
-  Score with the deployed model


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Work with service instance](#instance) 
3.	[Persist, deploy and score in an IBM Cloud](#scoring)
4.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener noreferrer">Watson Machine Learning Service</a> instance required for model storing and deployment (a free plan is offered)
-  **Not required on Watson Studio**: Install the <a href="https://pypi.python.org/pypi?%3Aaction=search&term=watson-machine-learning-client&submit=search" target="_blank" rel="noopener noreferrer">Watson-machine-learning-client</a> 

<a id="instance"></a>
## 2. Working with the instance

In this section you will learn how to get details of your instance.

Install python API client from pypi.

In [ ]:
!pip install watson-machine-learning-client --upgrade

**Tip:** If the above install fails due to `AssertionError: Multiple .dist-info ...`, uninstall the client library and remove the build directory with the commands below. 

Then run the `!pip install` again.

In [9]:
#!rm -rf /tmp/<directory-id-found-in-error>/pip-build/watson-machine-learning-client`

# where, for example, directory-id is: `s130-c0573632d58e90-35c9a7ecf27a`

**Action**: Enter your Watson Machine Learning service credentials from your IBM Cloud instance here.

In [11]:
wml_credentials = {
  "url": "***",
  "access_key": "***",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

In [12]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [13]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

Details about your Watson Machine Learning service are available in `instance_details`.

In [ ]:
print(wml_client.instance_details)

Get a list of the stored models for your WML instance:

In [15]:
wml_client.repository.list_models()

------------------------------------  ---------------------------------  ------------------------  ---------  -----
GUID                                  NAME                               CREATED                   FRAMEWORK  TYPE
c19ea489-1674-4883-bcbd-68f0089ef7dc  Customer Churn Model               2018-02-19T15:01:13.485Z  mllib-2.0  model
d9083d24-67f6-418e-869d-4fe344f7d9e5  Customer Churn Model               2018-02-20T07:20:03.989Z  mllib-2.1  model
7bce0af2-3d3a-4163-9bce-f5303c729d07  Customer Churn Model               2018-02-20T09:13:27.542Z  mllib-2.1  model
1698755c-ae01-4ab9-80b3-4f5d4d92bcc3  VIOLATIONS_SCALA211_SPARK20        2018-02-20T13:25:01.431Z  mllib-2.1  model
5f90e9ff-a9b3-46ad-bcc4-87d1ad6c9e7b  WML Product Line Prediction Model  2018-03-07T09:34:44.756Z  mllib-2.1  model
6765fc11-7c92-4132-90b4-b287a664d80e  WML Product Line Prediction Model  2018-03-07T09:50:24.003Z  mllib-2.1  model
c6f5af23-37fb-4e5f-b00e-76d87e3b7b34  WML Product Line Prediction Model  

Get a list of the deployed models for your WML instance:

In [16]:
wml_client.deployments.list()

------------------------------------  -------------------------  ------  ------------------------  ---------
GUID                                  NAME                       TYPE    CREATED                   FRAMEWORK
0c198739-0366-4733-96f5-0555e06d1390  Online scoring             online  2018-03-07T10:19:32.512Z  mllib-2.1
bb3cb635-7d08-4115-9592-113ff94aabde  Customer Churn Prediction  batch   2018-02-19T15:11:40.938Z  mllib-2.0
e3ee585e-d514-42cc-ba6a-74f1ff27a2b5  Online scoring             online  2018-03-07T09:37:09.140Z  mllib-2.1
ff347d25-500c-4adb-bd4c-96e73cba521b  Online scoring             online  2018-03-07T09:50:41.518Z  mllib-2.1
------------------------------------  -------------------------  ------  ------------------------  ---------


<a id="scoring"></a>
## 3. Persist, deploy and score model in a Cloud

In this section you will use the python API client for the Watson Machine Learning service to learn how to create an online deployment and to score a new data record.
For more information, see the <a href="http://wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener noreferrer">Python API client documentation</a>.

### 3.1: Persist model

**Action**: Download the sample PMML model from the git project using `wget`.

In [18]:
import wget, os

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store the downloaded file in the Watson Machine Learning repository.

In [19]:
props_pmml = {wml_client.repository.ModelMetaNames.NAME: 'CHAID PMML model for Iris data',
              wml_client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
              wml_client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: "pmml", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: "4.2"}

model_details = wml_client.repository.store_model(filename, props_pmml)
wml_client.repository.list_models()

------------------------------------  ---------------------------------  ------------------------  ---------  -----
GUID                                  NAME                               CREATED                   FRAMEWORK  TYPE
c19ea489-1674-4883-bcbd-68f0089ef7dc  Customer Churn Model               2018-02-19T15:01:13.485Z  mllib-2.0  model
d9083d24-67f6-418e-869d-4fe344f7d9e5  Customer Churn Model               2018-02-20T07:20:03.989Z  mllib-2.1  model
7bce0af2-3d3a-4163-9bce-f5303c729d07  Customer Churn Model               2018-02-20T09:13:27.542Z  mllib-2.1  model
1698755c-ae01-4ab9-80b3-4f5d4d92bcc3  VIOLATIONS_SCALA211_SPARK20        2018-02-20T13:25:01.431Z  mllib-2.1  model
5f90e9ff-a9b3-46ad-bcc4-87d1ad6c9e7b  WML Product Line Prediction Model  2018-03-07T09:34:44.756Z  mllib-2.1  model
6765fc11-7c92-4132-90b4-b287a664d80e  WML Product Line Prediction Model  2018-03-07T09:50:24.003Z  mllib-2.1  model
c6f5af23-37fb-4e5f-b00e-76d87e3b7b34  WML Product Line Prediction Model  

**Note:** You can see that the model is successfully stored in the Watson Machine Learning Service.

### 3.2: Create online deployment

Use the commands below to create the online deployment for the stored model.

In [20]:
model_uid = wml_client.repository.get_model_uid(model_details)
model_deployment = wml_client.deployments.create(model_uid, name = 'Iris species prediction')

wml_client.deployments.list()

------------------------------------  -------------------------  ------  ------------------------  ---------
GUID                                  NAME                       TYPE    CREATED                   FRAMEWORK
0c198739-0366-4733-96f5-0555e06d1390  Online scoring             online  2018-03-07T10:19:32.512Z  mllib-2.1
25df3a17-ae3b-44e8-acb0-8943f6b75a61  Iris species prediction    online  2018-03-08T12:37:25.062Z  pmml-4.2
bb3cb635-7d08-4115-9592-113ff94aabde  Customer Churn Prediction  batch   2018-02-19T15:11:40.938Z  mllib-2.0
e3ee585e-d514-42cc-ba6a-74f1ff27a2b5  Online scoring             online  2018-03-07T09:37:09.140Z  mllib-2.1
ff347d25-500c-4adb-bd4c-96e73cba521b  Online scoring             online  2018-03-07T09:50:41.518Z  mllib-2.1
------------------------------------  -------------------------  ------  ------------------------  ---------


### 3.3: Scoring

Check the online scoring endpoint by sending a sample record using the python API client.

In [21]:
model_scoring_url = wml_client.deployments.get_scoring_url(model_deployment)

print(model_scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/8f11cfac-57ca-4a0c-9815-7abbd7a72491/published_models/789cde6f-464a-45ce-b159-36619be36ec3/deployments/25df3a17-ae3b-44e8-acb0-8943f6b75a61/online


You can see that the scoring url is available for sending scoring requests.

In [22]:
scoring_data = {'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'], 'values': [[5.1, 3.5, 1.4, 0.2]]}
predictions = wml_client.deployments.score(model_scoring_url, scoring_data) 

print(predictions)

{'fields': ['PredictedValue', 'Confidence'], 'values': [['setosa', 1.0]]}


As we can see this is the Iris Setosa flower.

<a id="summary"></a>
## 4. Summary and next steps     

You successfully completed this notebook! You learned how to use Watson Machine Learning for PMML model deployment and scoring. Check out our <a href="https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html?pos=2" target="_blank" rel="noopener noreferrer">Online Documentation</a> (for more samples, tutorials, documentation, how-tos, and blog posts). 

### Citations

Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science. 

### Authors

**Wojciech Sobala**, Data Scientist in IBM.

Copyright © 2018 IBM. This notebook and its source code are released under the terms of the MIT License.